In [1]:
!pip install serpapi langchain_community wikipedia google-search-results amadeus

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=e5966f1b812982d1816f0db0db17c1ddefe1483e84e497c183571eac6882e2a9
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=8894a20a250b67a8138309e69c6efb50cbf26b6fab90dc82df3f6ee6e2194242
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
  Created wheel for amadeus: filename=amadeus-11.0.0-py2.py3-none-any.whl size=71670 sha256=ae5

In [2]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [13]:
###########################################################################################################

# Import necessary libraries
import os
import requests
import re
import json
import warnings
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage
from serpapi import GoogleSearch
from amadeus import Client, ResponseError

# Remove warning messages
warnings.filterwarnings("ignore")

###########################################################################################################

# Set up API keys
os.environ["OPENAI_API_KEY"] = "##############"  # Replace with your API Keys
os.environ["AMADEUS_API_KEY"] = "#############"
os.environ["AMADEUS_API_SECRET"] = "###########"
google_maps_api_key = "###########"
weather_api_key = "############"
serpapi_key = "############"  
geoapify_api_key = "############"

# Initialize Amadeus API client
amadeus = Client(
    client_id=os.getenv("AMADEUS_API_KEY"),
    client_secret=os.getenv("AMADEUS_API_SECRET")
)

# Initialize OpenAI Model
llm = ChatOpenAI(model="gpt-4", temperature = 0, openai_api_key=os.getenv("OPENAI_API_KEY"))

###########################################################################################################

# Function to fetch latitude and longitude of a location ( Used for fetching Hotels info, Tourist locations info)
def get_lat_lng(location, google_maps_api_key):
    """
    Fetch latitude and longitude for a given location using Google Maps Geocoding API.
    """
    if not location:
        return None, None  # Ensure no invalid assumptions

    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        "address": location,
        "key": google_maps_api_key
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Handle HTTP errors

        # Parse response
        data = response.json()
        if "results" in data and len(data["results"]) > 0:
            location_data = data["results"][0]["geometry"]["location"]
            return location_data["lat"], location_data["lng"]
        else:
            print(f"No geolocation results found for '{location}'.")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching geolocation for '{location}': {e}")
        return None, None

###########################################################################################################

def fetch_restaurants(location, google_maps_api_key, radius=50000, top_n=5):
    """
    Fetch restaurants near the specified location using Google Maps Places API.
    """
    # Get latitude and longitude for the location
    lat, lng = get_lat_lng(location, google_maps_api_key)
    if not lat or not lng:
        return f"Could not determine the exact location for '{location}'. Please provide a valid location."

    # URL for Google Maps Places API
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{lat},{lng}",
        "radius": radius,
        "type": "restaurant",
        "key": google_maps_api_key
    }

    try:
        # Make the request to the Google Maps Places API
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an error for HTTP issues
        data = response.json()

        # Parse the results
        if "results" in data and len(data["results"]) > 0:
            restaurants = data["results"]
            result = []
            for restaurant in restaurants[:top_n]:
                name = restaurant.get("name", "Name not available")
                rating = restaurant.get("rating", "No rating")
                address = restaurant.get("vicinity", "Address not available")
                result.append(
                    f"Restaurant: {name}\nRating: {rating}\nAddress: {address}\n"
                    "----------------------------------------"
                )
            return "\n\n".join(result)
        else:
            return f"No restaurants found near '{location}'."
    except requests.exceptions.RequestException as e:
        return f"Error fetching restaurants: {e}"

###########################################################################################################

# Function to fetch Tourist Places for a location
def fetch_tourist_places(location, google_maps_api_key, serpapi_key, radius=50000, top_n=5):
    """
    Fetch tourist places near the specified location using SERP API.
    """
    # Get latitude and longitude for the location
    lat, lng = get_lat_lng(location, google_maps_api_key)
    # Validate the input location
    if not location:
        return "Please specify a location to search for tourist places."


    if not lat or not lng:
        return f"Could not determine the exact location for '{location}'. Please provide a valid location."

    # SERP API query parameters
    params = {
        "engine": "google_maps",
        "q": "tourist attractions",
        "ll": f"@{lat},{lng},15z",  # Latitude and Longitude
        "radius": radius,
        "hl": "en",
        "api_key": serpapi_key
    }

    try:
        # Fetch data from SERP API
        search = GoogleSearch(params)
        results = search.get_dict()

        # Check if local_results is valid
        if "local_results" in results and isinstance(results["local_results"], list):
            places = results["local_results"]
            result = []
            for place in places[:top_n]:
                name = place.get("title", "Name not available")
                rating = place.get("rating", "No rating")
                address = place.get("address", "Address not available")
                description = place.get("description", "Description not available")
                result.append(
                    f"Place: {name}\nRating: {rating}\nAddress: {address}\nDescription: {description}\n"
                    "----------------------------------------"
                )
            return "\n\n".join(result)
        else:
            return f"No tourist places found near '{location}'."
    except Exception as e:
        return f"Error fetching tourist places: {e}"

###########################################################################################################

# Function to fetch history of each tourist location
def search_wikipedia_page(place_name):
    """
    Search for a Wikipedia page related to the given place name.
    """
    search_url = f"https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": place_name,
        "format": "json",
    }
    try:
        response = requests.get(search_url, params=params)
        response.raise_for_status()
        data = response.json()
        if data["query"]["search"]:
            # Return the title of the first result
            return data["query"]["search"][0]["title"]
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error searching Wikipedia: {e}")
        return None

def fetch_place_brief_summary(place_name):
    """
    Fetch a brief historical summary of a place from Wikipedia.
    """
    # Search for a Wikipedia page
    page_title = search_wikipedia_page(place_name)
    if not page_title:
        return f"No brief information available for {place_name}."

    # Format the page title for the summary API
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{page_title.replace(' ', '_')}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if 'extract' in data:
            summary = data["extract"].split(". ")
            return ". ".join(summary[:2]) + "."
    except requests.exceptions.RequestException as e:
        print(f"Error fetching summary for {place_name}: {e}")
        return f"No brief information available for {place_name}."
    return f"No brief information available for {place_name}."

###########################################################################################################

# Function to fetch hotels for a location
def fetch_hotels(location, top_n=5, max_price=None):
    lat, lng = get_lat_lng(location, google_maps_api_key)
    if not lat or not lng:
        return "Could not determine the exact location."
    # Validate the input location
    if not location:
        return "Please specify a location to search for hotels."
    params = {
        "engine": "google_maps",
        "ll": f"@{lat},{lng},14z",
        "q": "hotels",
        "radius": 50000,
        "hl": "en",
        "api_key": serpapi_key
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    if "local_results" in results:
        hotels = results["local_results"]
        result = []
        for hotel in hotels[:top_n]:
            name = hotel.get("title", "Name not available")
            rating = hotel.get("rating", "No rating")
            address = hotel.get("address", "Address not available")
            price = hotel.get("price", "Price not available")
            description = hotel.get("description", "Description not available")
            result.append(
                f"Hotel: {name}\nRating: {rating}\nAddress: {address}\nPrice: {price}\nDescription: {description}\n"
                "----------------------------------------"
            )
        return "\n\n".join(result)
    return "No hotel data found in the specified area."

###########################################################################################################

# Function to fetch weather for a location
def fetch_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": weather_api_key,
        "units": "metric"
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        weather_data = response.json()
        description = weather_data["weather"][0]["description"]
        temp = weather_data["main"]["temp"]
        return f"The weather in {city} is {description} with a temperature of {temp}°C."
    except requests.exceptions.RequestException:
        return f"Could not fetch weather data for {city}."

###########################################################################################################

# Helper Function to format ISO 8601 duration (converts 'Duration' to human understanding format)
def format_duration(iso_duration):
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?', iso_duration)
    hours = match.group(1) if match.group(1) else "0"
    minutes = match.group(2) if match.group(2) else "0"
    return f"{int(hours)} hours {int(minutes)} minutes"

# Helper Function to get full airline names from its codes using OpenAI
def get_airline_full_name(airline_code):
    prompt = f"Please provide only the full name for the airline '{airline_code}'."
    response = llm([HumanMessage(content=prompt)])
    return response.content.strip() if response else airline_code  # Return the code if response is empty

# Function to fetch Flights information
def fetch_flights(origin, destination, departure_date, return_date=None, max_price=None, airline_name =None):
    try:
        # Set a high default max_price if not provided
        max_price = max_price if max_price else 20000
        params = {
            "originLocationCode": origin,
            "destinationLocationCode": destination,
            "departureDate": departure_date,
            "adults": 1,
            "maxPrice": max_price
        }

        if return_date:
            params["returnDate"] = return_date  # Include return date for round-trip flights

        # Fetch flights from Amadeus API
        response = amadeus.shopping.flight_offers_search.get(**params)
        flights = response.data

        if flights:
            result = []
            for flight in flights[:5]:  # Limit to top 5 results
                if float(flight['price']['total']) <= max_price:
                    # Outbound flight details
                    segments = flight['itineraries'][0]['segments']
                    airline_code = segments[0]['carrierCode']
                    airline = get_airline_full_name(airline_code)  # Get full airline name
                    # Only add flights that match the specified airline, if provided
                    if airline_name and airline and airline.lower() not in airline_name.lower():
                        continue
                    departure_time = segments[0]['departure']['at']
                    arrival_time = segments[-1]['arrival']['at']
                    flight_duration = format_duration(flight['itineraries'][0]['duration'])

                    # Only include return details if a return date is provided
                    if return_date and len(flight['itineraries']) > 1:
                        return_segments = flight['itineraries'][1]['segments']
                        return_departure_time = return_segments[0]['departure']['at']
                        return_arrival_time = return_segments[-1]['arrival']['at']
                        return_duration = format_duration(flight['itineraries'][1]['duration'])
                        return_info = (
                            f"\nReturn Departure: {return_departure_time}\n"
                            f"Return Arrival: {return_arrival_time}\n"
                            f"Return Duration: {return_duration}\n"
                        )
                    else:
                        return_info = ""

                    # Append both outbound and return information (if available) to results
                    result.append(
                        f"Airline: {airline}\nPrice: ${flight['price']['total']}\n"
                        f"Departure: {departure_time}\nArrival: {arrival_time}\n"
                        f"Duration: {flight_duration}{return_info}"
                        "\n----------------------------------------"
                    )
            return "\n\n".join(result) if result else "No flights found within the budget."
        return "No flights found."
    except ResponseError as error:
        return f"An error occurred: {error.response.result}"

# Helper function to extract flight details from user query
def parse_flight_details(query):
    prompt = (
        f"Extract the following details from this flight request:\n"
        f"Origin location (IATA code if possible), Destination location (IATA code if possible), "
        f"Departure date in YYYY-MM-DD format, and Return date if provided.\n\n"
        f"Request: '{query}'\n\n"
        f"Respond in JSON format with 'origin', 'destination', 'departure_date', and 'return_date'. "
        f"Use IATA codes for origin and destination if known, otherwise, use city names."
    )

    message = HumanMessage(content=prompt)           # Create a HumanMessage object from the user query
    response = llm([message])                        # Pass the HumanMessage to the LLM and get the response
    response_text = response.content.strip()

    try:
        # Attempt to parse the response as JSON
        details = json.loads(response_text)
    except json.JSONDecodeError:
        print("Error: Unable to parse response as JSON.")
        raise ValueError("Invalid JSON response from model.")

    origin = details.get("origin")
    destination = details.get("destination")
    departure_date = details.get("departure_date")
    return_date = details.get("return_date")

    if not all([origin, destination, departure_date]):
        raise ValueError("Incomplete flight details extracted from the query.")

    return origin, destination, departure_date, return_date

###########################################################################################################

def public_transportation(location, geoapify_api_key, radius=1000, limit=5):
    """
    Fetch transportation information near a given location using Geoapify Places API.

    Parameters:
    - location: Address or name of the location (string)
    - geoapify_api_key: Geoapify API key (string)
    - radius: Search radius in meters (default is 1000 meters)
    - limit: Maximum number of results to return (default is 10)

    Returns:
    - A list of transport hubs near the location with details, or an error message.
    """
    # Helper function to get latitude and longitude
    def lat_lng(location, geoapify_api_key):
        url = "https://api.geoapify.com/v1/geocode/search"
        params = {
            "text": location,
            "apiKey": geoapify_api_key
        }
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            if "features" in data and len(data["features"]) > 0:
                coords = data["features"][0]["geometry"]["coordinates"]
                return coords[1], coords[0]  # Return latitude, longitude
            else:
                return None, None
        except requests.exceptions.RequestException as e:
            print(f"Error fetching geolocation for '{location}': {e}")
            return None, None

    # Get latitude and longitude for the location
    lat, lng = lat_lng(location, geoapify_api_key)
    if not lat or not lng:
        return f"Could not determine the exact location for '{location}'. Please provide a valid location."

    # Use Geoapify Places API to search for transportation hubs
    url = "https://api.geoapify.com/v2/places"
    params = {
        "categories": "public_transport",  # Search for public transport hubs
        "filter": f"circle:{lng},{lat},{radius}",
        "limit": limit,
        "apiKey": geoapify_api_key
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        if "features" in data and len(data["features"]) > 0:
            transport_info = []
            for feature in data["features"]:
                name = feature["properties"].get("name", "Name not available")
                category = feature["properties"].get("categories", [])
                address = feature["properties"].get("formatted", "Address not available")
                transport_info.append(
                    f"Transport Hub: {name}\nCategories: {', '.join(category)}\nAddress: {address}\n"
                    "----------------------------------------"
                )
            return "\n\n".join(transport_info)
        else:
            return f"No transportation hubs found near '{location}'."
    except requests.exceptions.RequestException as e:
        return f"Error fetching transportation information: {e}"

###########################################################################################################

# Define tools for each agent
tools = [
    Tool(name="Hotel Search", func=fetch_hotels, description="Search for hotels at a specified location."),
    Tool(name="Tourist Places", func=lambda location: fetch_tourist_places(location, google_maps_api_key=google_maps_api_key, serpapi_key=serpapi_key), description="Find tourist places at a location."),
    Tool(name="Wikipedia History", func=fetch_place_brief_summary, description="Get history of a tourist place."),
    Tool(name="Weather", func=fetch_weather, description="Get current weather in a city."),
    Tool(name="Flight Search", func=lambda input_str: fetch_flights(
        **{k: v for k, v in json.loads(input_str).items() if k in ["origin", "destination", "departure_date", "return_date"]}
    ),
    description="Find flights based on origin, destination, departure date, and return date."),
    Tool(
    name="Restaurents",
    func=lambda location: fetch_restaurants(
        location, google_maps_api_key=google_maps_api_key,
    ),
    description="Find restaurants at a location."),
    Tool(
    name="Local Public Transport",
    func=lambda location: public_transportation(
        location, geoapify_api_key=geoapify_api_key,  # Replace with your Geoapify API key variable
    ),
    description="Fetch local transportation hubs (e.g., bus stops, train stations) near a specified location.")]

###########################################################################################################

# Initialize agent with all the tools for each agent
memory = ConversationBufferMemory()
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

###########################################################################################################

# Main function
def get_recommendations():
    user_query = input("Enter your request: ")       # Prompt the user to enter their request

    # Determine if the query is for flight recommendations
    if "flight" in user_query.lower():
        try:
            origin, destination, departure_date, return_date = parse_flight_details(user_query) # Parse the details for a flight request

            # Prepare structured flight details
            flight_details = {
                "origin": origin,
                "destination": destination,
                "departure_date": departure_date,
                "return_date": return_date
            }

            # Pass structured flight details to the agent
            response = agent.run(json.dumps(flight_details))
        except ValueError as e:
            print(f"Error parsing flight details: {e}")
            return
    else:
        # Pass the general query directly to the agent
        response = agent.run(user_query)

    print(response)

get_recommendations()  # Main Function call

###########################################################################################################

# Examples for User Queries:
# Recommend me tourist places and information regarding each of the tourist place in puducherry
# Recommend flights for round trip from Baltimore to Dallas on December24th, 2024 and return on January 4th, 2025 within $500
# Find hotels in Boston within $250 per day
# What is the weather condition at Palakkad, Kerala?
# What are the tourist places with higher ratings in Coimbatore, Tamilnadu?
# Recommend flights from IAD to HYD on december, 2024 within $600
# Recommend restaurents with biryani in lb nagar, hyderabad
# Recommend me public transportation in vanasthalipuram, hyderabad
# What is the address of metro railway station near royapuram, chennai
# Recommend me spriritual tourist places in hyderabad


Enter your request: address of metro railway station near royapuram, chennai


> Entering new AgentExecutor chain...
I need to find the local public transport near the specified location.
Action: Local Public Transport
Action Input: royapuram, chennai
Observation: Transport Hub: Royapuram
Categories: building, building.historic, building.transportation, public_transport, public_transport.train, tourism, tourism.sights
Address: North Terminus Road, Zone 5 Royapuram, Chennai - 600001, Tamil Nadu, India
----------------------------------------

Transport Hub: Clive Battery
Categories: public_transport, public_transport.bus
Address: Clive Battery, Rajaji Salai, Zone 5 Royapuram, Chennai - 600001, Tamil Nadu, India
----------------------------------------

Transport Hub: Name not available
Categories: public_transport, public_transport.bus
Address: Broadway Road, Zone 5 Royapuram, Chennai - 600001, Tamil Nadu, India
----------------------------------------

Transport Hub: Name not available

In [4]:
#print(agent.run("Recommend me tourist places and information regarding each of the tourist place in puducherry"))

In [5]:
#print(agent.run("Find hotels in Boston within $250 per day"))

In [6]:
#print(agent.run("What is the weather condition at Palakkad, Kerala?"))

In [7]:
#print(agent.run("What are the tourist places with higher ratings in Coimbatore, Tamilnadu?"))

In [8]:
#print(agent.run("Recommend flights from IAD to HYD on december, 2024 within $600"))